<a href="https://colab.research.google.com/github/beruscoder/Bhatiyani-Astute-Intelligence-assignment/blob/main/colab_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U ultralytics supervision opencv-python-headless -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 7.6 MB/s eta 0:00:00


In [2]:
# ==============================================================
# Footfall Counter with ROI Polygon + Trajectories + Heatmap
# (YOLOv8 + ByteTrack + Supervision)
# --------------------------------------------------------------
# Author : Jansty Lewis
# Description : Detect, track, and count people entering/exiting
#               a defined Region of Interest (ROI) using YOLOv8.
# ==============================================================

# Install dependencies (uncomment if running in Colab)
# !pip install -U ultralytics supervision opencv-python-headless -q

# --------------------------------------------------------------
# Imports
# --------------------------------------------------------------
import time
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
import supervision as sv


# ==============================================================
# 1️⃣ Helper Class — Centroid Smoothing
# --------------------------------------------------------------
# Smoothens object centroids to reduce flickering and noise
# when computing trajectories and heatmaps.
# ==============================================================
class CentroidSmoother:
    def __init__(self, alpha=0.6):
        """
        alpha : float
            Smoothing factor (0 < alpha <= 1).
            Higher alpha = faster response, less smoothing.
        """
        self.alpha = alpha
        self.states = {}  # Stores smoothed coordinates {id: (x, y)}

    def smooth(self, tid, cx, cy):
        """Applies exponential moving average smoothing to centroid."""
        if tid not in self.states:
            self.states[tid] = (cx, cy)
            return cx, cy

        px, py = self.states[tid]
        nx = int(self.alpha * cx + (1 - self.alpha) * px)
        ny = int(self.alpha * cy + (1 - self.alpha) * py)
        self.states[tid] = (nx, ny)
        return nx, ny


# ==============================================================
# 2️⃣ Utility Functions
# --------------------------------------------------------------
# Contains helper logic for counting, drawing, and visualization.
# ==============================================================

def define_roi_polygon(out_w, out_h):
    """Define polygonal Region of Interest (ROI). Modify as needed."""
    polygon = np.array([
        [150, int(out_h * 0.7)],
        [out_w - 150, int(out_h * 0.7)],
        [out_w - 150, out_h - 20],
        [150, out_h - 20]
    ])
    return polygon


def update_counting_state(
    tid, sx, sy, polygon, last_state, last_count_time,
    count_in, count_out, count_cooldown
):
    """
    Determines whether an object has entered or exited the ROI
    and updates counts accordingly.
    """
    inside = cv2.pointPolygonTest(polygon, (sx, sy), False) >= 0
    prev_state = last_state.get(tid, "outside")
    current_state = "inside" if inside else "outside"
    now = time.time()

    if prev_state != current_state:
        # Check cooldown to avoid multiple triggers
        if now - last_count_time[tid] > count_cooldown:
            if prev_state == "outside" and current_state == "inside":
                count_in += 1
            elif prev_state == "inside" and current_state == "outside":
                count_out += 1
            last_count_time[tid] = now

    last_state[tid] = current_state
    return current_state, count_in, count_out


def draw_trajectories(frame, traces):
    """Draw trajectory lines for each tracked object."""
    for tid, pts in traces.items():
        if len(pts) > 1:
            for i in range(1, len(pts)):
                cv2.line(frame, pts[i - 1], pts[i], (0, 255, 255), 2)
    return frame


def overlay_heatmap(base_frame, heatmap):
    """Generate and overlay the heatmap on top of the base frame."""
    blur = cv2.GaussianBlur(heatmap, (35, 35), 0)
    norm = cv2.normalize(blur, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    colored = cv2.applyColorMap(norm, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(base_frame, 0.8, colored, 0.4, 0)
    return overlay


def draw_info_overlay(frame, count_in, count_out, active_inside):
    """Displays live statistics (In/Out/Active) on the frame."""
    cv2.putText(frame, f"In: {count_in}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(frame, f"Out: {count_out}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(frame, f"Active (inside): {active_inside}", (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    return frame


# ==============================================================
# 3️⃣ Main Function — Tracking + Counting + Visualization
# ==============================================================
def track_and_count_persons_roi(
    source="mall.mp4",
    output_path="output_person_roi.avi",
    model_name="yolov8s.pt",
    resize_width=None,
    conf=0.35,
    classes=[0],  # Person class
):
    """
    Core function to perform:
    - Object detection using YOLOv8
    - Tracking using ByteTrack
    - ROI-based entry/exit counting
    - Trajectory and heatmap visualization
    """
    # ------------------- Initialization ------------------------
    model = YOLO(model_name)
    tracker = sv.ByteTrack()
    box_annotator = sv.BoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator()

    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {source}")

    W, H = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0

    # Resize settings
    if resize_width is not None:
        scale = resize_width / float(W)
        out_w, out_h = int(resize_width), int(H * scale)
    else:
        out_w, out_h, scale = W, H, 1.0

    # Output video writer
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    writer = cv2.VideoWriter(output_path, fourcc, fps, (out_w, out_h))

    # ------------------- ROI & Annotations ----------------------
    polygon = define_roi_polygon(out_w, out_h)
    zone = sv.PolygonZone(polygon=polygon)
    zone_annotator = sv.PolygonZoneAnnotator(
        zone=zone, color=sv.Color.from_hex("#0000FF"),
        thickness=2, text_thickness=2, text_scale=1
    )

    # ------------------- Tracking State -------------------------
    smoother = CentroidSmoother(alpha=0.6)
    traces = defaultdict(list)
    heatmap = np.zeros((out_h, out_w), dtype=np.float32)

    last_state = {}
    last_count_time = defaultdict(lambda: -9999)
    count_cooldown = 0.8
    count_in, count_out = 0, 0

    frame_idx, start_ts = 0, time.time()

    # ==========================================================
    # Frame-by-Frame Processing
    # ==========================================================
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_idx += 1

            # Resize (if required)
            frame_resized = cv2.resize(frame, (out_w, out_h)) if resize_width else frame.copy()

            # ---------------- Detection & Tracking ----------------
            results = model.predict(source=[frame_resized], imgsz=640, conf=conf, classes=classes, verbose=False)
            detections = sv.Detections.from_ultralytics(results[0])
            tracked = tracker.update_with_detections(detections)

            # ---------------- Visualization ------------------------
            labels = [f"person {int(tid)}" for tid in getattr(tracked, "tracker_id", [])]
            annotated = box_annotator.annotate(scene=frame_resized.copy(), detections=tracked)
            if labels:
                annotated = label_annotator.annotate(scene=annotated, detections=tracked, labels=labels)
            annotated = zone_annotator.annotate(scene=annotated)

            # ---------------- Counting Logic -----------------------
            current_inside = set()
            tracked_xyxy, tracked_ids = getattr(tracked, "xyxy", []), getattr(tracked, "tracker_id", [])

            if tracked_xyxy is not None and len(tracked_xyxy) > 0:
                for bb, tid in zip(tracked_xyxy, tracked_ids):
                    x1, y1, x2, y2 = bb
                    cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
                    sx, sy = smoother.smooth(int(tid), cx, cy)
                    traces[tid].append((sx, sy))
                    cv2.circle(heatmap, (sx, sy), 8, 1, -1)

                    current_state, count_in, count_out = update_counting_state(
                        tid, sx, sy, polygon, last_state, last_count_time,
                        count_in, count_out, count_cooldown
                    )

                    if current_state == "inside":
                        current_inside.add(tid)

            active_inside = len(current_inside)

            # ---------------- Overlays -----------------------------
            annotated = draw_trajectories(annotated, traces)
            overlay = overlay_heatmap(annotated, heatmap)
            overlay = draw_info_overlay(overlay, count_in, count_out, active_inside)

            # Write frame
            writer.write(overlay)

            # Progress log
            if frame_idx % 100 == 0:
                elapsed = time.time() - start_ts
                print(f"Processed {frame_idx} frames | In:{count_in} Out:{count_out} Active:{active_inside} | Time: {elapsed:.1f}s")

    finally:
        cap.release()
        writer.release()
        print(f"✅ Finished. Final counts => In: {count_in}, Out: {count_out}")
        print(f"🎥 Output saved to: {output_path}")


# ==============================================================
# 4️⃣ Example Entry Point (For Google Colab)
# ==============================================================
if __name__ == "__main__":
    track_and_count_persons_roi(
        source="/content/walking.mp4",
        output_path="/content/output_person_roi.avi",
        model_name="yolov8s.pt",
        resize_width=640,
        conf=0.35
    )


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Processed 100 frames | In:5 Out:1 Active:4 | Time: 6.3s
Processed 200 frames | In:8 Out:5 Active:3 | Time: 10.9s
Processed 300 frames | In:12 Out:10 Active:2 | Time: 16.2s
✅ Finished. Final counts => In: 16, Out: 12
🎥 Output saved to: /content/output_person_roi.avi
